In [10]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Conv1D, BatchNormalization, LeakyReLU, MaxPool1D, GlobalAveragePooling1D, Dense, Dropout, AveragePooling1D, Input
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
Data_array = np.transpose(feet['data_array'], (2, 0, 1))
Label_array = feet['label_array']

def cnnmodel():
    model = Sequential()
    model.add(Input(shape=(19, 300)))  # Define input shape explicitly
    model.add(Conv1D(filters=32, kernel_size=5, strides=1, padding='same'))  # 1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))  # 2
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same'))  # 3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2, strides=2))  # 4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))  # 5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding='same'))  # 7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 8
    model.add(Conv1D(filters=512, kernel_size=3, strides=1, padding='same'))  # 9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())  # 10
    model.add(Dense(1, activation='sigmoid'))  # 11

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnnmodel()
model.summary()

# Define KFold cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracy = []
for train_index, val_index in kf.split(Data_array):
    train_features, train_labels = Data_array[train_index], Label_array[train_index]
    val_features, val_labels = Data_array[val_index], Label_array[val_index]
    
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features.reshape(train_features.shape[0], -1)).reshape(train_features.shape)
    val_features_scaled = scaler.transform(val_features.reshape(val_features.shape[0], -1)).reshape(val_features.shape)
    
    model = cnnmodel()
    model.fit(train_features_scaled, train_labels, epochs=50, batch_size=128, validation_data=(val_features_scaled, val_labels))
    
    # Evaluate accuracy on validation set
    val_pred = (model.predict(val_features_scaled) > 0.5).astype(int)
    acc = accuracy_score(val_labels, val_pred)
    accuracy.append(acc)

# Print mean accuracy across folds
mean_accuracy = np.mean(accuracy)
print("Mean Validation Accuracy:", mean_accuracy)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_19 (Conv1D)              │ (None, 19, 32)         │        48,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 19, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_18 (LeakyReLU)      │ (None, 19, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 9, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 9, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_19 (LeakyReLU)      │ (None, 9, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 4, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_20 (LeakyReLU)      │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_7             │ (None, 2, 128)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 2, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_21 (LeakyReLU)      │ (None, 2, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_8             │ (None, 1, 256)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_23 (Conv1D)              │ (None, 1, 512)         │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_22 (LeakyReLU)      │ (None, 1, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 571,873 (2.18 MB)

 Trainable params: 571,809 (2.18 MB)

 Non-trainable params: 64 (256.00 B)

Epoch 1/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.6095 - loss: 0.6741 - val_accuracy: 0.6095 - val_loss: 0.6636
Epoch 2/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6232 - loss: 0.6634 - val_accuracy: 0.6095 - val_loss: 0.6670
Epoch 3/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6222 - loss: 0.6609 - val_accuracy: 0.6095 - val_loss: 0.6657
Epoch 4/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6150 - loss: 0.6615 - val_accuracy: 0.6095 - val_loss: 0.6647
Epoch 5/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6204 - loss: 0.6577 - val_accuracy: 0.5652 - val_loss: 0.6843
Epoch 6/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6268 - loss: 0.6483 - val_accuracy: 0.5986 - val_loss: 0.6732
Epoch 7/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6304 - loss: 0.6362 - val_accuracy: 0.5968 - val_loss: 0.6756
Epoch 8/50
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6541 - loss: 0.6191 - val_accuracy: 0.5765 - v

In [16]:
Data_array = np.moveaxis(Data_array,2,1)
Data_array.shape

(11072, 300, 19)

In [17]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical


model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(300, 19)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')  # 이진 분류를 위한 출력층
])

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

# 모델 학습
model.fit(Data_array, Label_array, epochs=10, batch_size=64, validation_split=0.2)


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_51 (Conv1D)              │ (None, 298, 64)        │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 149, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_52 (Conv1D)              │ (None, 147, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_23 (MaxPooling1D) │ (None, 73, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9344)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │       598,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 626,626 (2.39 MB)

 Trainable params: 626,626 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 2)

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling1D, AveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming Data_array and Label_array are already defined
# Splitting the data into training and validation sets
Data_array = np.transpose(feet['data_array'], (2, 0, 1))  # Transpose the data array

X_train, X_val, y_train, y_val = train_test_split(Data_array, Label_array, test_size=0.2, random_state=42)

def cnnmodel():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=5, strides=1, padding='same', input_shape=(19, 300)))  # 1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))  # 2
    model.add(Conv1D(filters=64, kernel_size=3, strides=1, padding='same'))  # 3
    model.add(LeakyReLU())
    model.add(MaxPooling1D(pool_size=2, strides=2))  # 4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=128, kernel_size=3, strides=1, padding='same'))  # 5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=256, kernel_size=3, strides=1, padding='same'))  # 7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2, strides=2))  # 8
    model.add(Conv1D(filters=512, kernel_size=3, strides=1, padding='same'))  # 9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())  # 10
    model.add(Dense(1, activation='sigmoid'))  # 11

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnnmodel()

# Model summary
model.summary()

# Model training
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the model on the validation data
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_67 (Conv1D)              │ (None, 19, 32)         │        48,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 19, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_58 (LeakyReLU)      │ (None, 19, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_32 (MaxPooling1D) │ (None, 9, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_68 (Conv1D)              │ (None, 9, 64)          │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_59 (LeakyReLU)      │ (None, 9, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_33 (MaxPooling1D) │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_69 (Conv1D)              │ (None, 4, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_60 (LeakyReLU)      │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_23            │ (None, 2, 128)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_70 (Conv1D)              │ (None, 2, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_61 (LeakyReLU)      │ (None, 2, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_24            │ (None, 1, 256)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_71 (Conv1D)              │ (None, 1, 512)         │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_62 (LeakyReLU)      │ (None, 1, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_11     │ (None, 512)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 571,873 (2.18 MB)

 Trainable params: 571,809 (2.18 MB)

 Non-trainable params: 64 (256.00 B)

Epoch 1/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6003 - loss: 0.6811 - val_accuracy: 0.6054 - val_loss: 0.6725
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6212 - loss: 0.6651 - val_accuracy: 0.6095 - val_loss: 0.6710
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6251 - loss: 0.6622 - val_accuracy: 0.6095 - val_loss: 0.6697
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6303 - loss: 0.6536 - val_accuracy: 0.6095 - val_loss: 0.6709
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6250 - loss: 0.6549 - val_accuracy: 0.6095 - val_loss: 0.6713
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6240 - loss: 0.6517 - val_accuracy: 0.6099 - val_loss: 0.6689
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6279 - loss: 0.6457 - val_accuracy: 0.6063 - val_loss: 0.6758
Epoch 8/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6345 - loss: 0.6402 - va

In [31]:
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU, MaxPooling1D, Conv1D, Dropout, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

# Assuming Data_array and Label_array are already defined
Data_array = np.transpose(feet['data_array'], (2, 0, 1))  # Transpose the data array
Label_array = feet['label_array']

# Reshape data to have the correct input shape for Conv1D
num_samples = Data_array.shape[0]
num_channels = Data_array.shape[1]
time_steps = Data_array.shape[2]

# Reshape data to match Conv1D input shape: (num_samples, time_steps, num_channels)
X = Data_array.reshape(num_samples, time_steps, num_channels)

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, Label_array, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(-1, time_steps * num_channels), y_train)

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)

def cnnmodel():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(time_steps, num_channels)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnnmodel()

# Model summary
model.summary()

# Model training with class weights
history = model.fit(X_train_resampled.reshape(-1, time_steps, num_channels), y_train_resampled, epochs=100, batch_size=64,
                    validation_data=(X_val, y_val), class_weight=dict(enumerate(class_weights)))

# Evaluate the model on the validation data
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_77 (Conv1D)              │ (None, 298, 64)        │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_38 (MaxPooling1D) │ (None, 149, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_78 (Conv1D)              │ (None, 147, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_39 (MaxPooling1D) │ (None, 73, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 9344)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 64)             │       598,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_34 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 626,561 (2.39 MB)

 Trainable params: 626,561 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.4926 - loss: 2.0144 - val_accuracy: 0.3959 - val_loss: 0.6947
Epoch 2/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5167 - loss: 0.6916 - val_accuracy: 0.3901 - val_loss: 0.6958
Epoch 3/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5061 - loss: 0.6931 - val_accuracy: 0.3901 - val_loss: 0.6946
Epoch 4/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5026 - loss: 0.6931 - val_accuracy: 0.3919 - val_loss: 0.6941
Epoch 5/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.5311 - loss: 0.6882 - val_accuracy: 0.4682 - val_loss: 0.6958
Epoch 6/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5795 - loss: 0.6725 - val_accuracy: 0.4844 - val_loss: 0.6961
Epoch 7/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5970 - loss: 0.6558 - val_accuracy: 0.5436 - val_loss: 0.6821
Epoch 8/100
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.6284 - loss: 0.6300 - 

In [2]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample
from tensorflow.keras.callbacks import EarlyStopping

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_points, channels)
X = np.transpose(data_array, (2, 1, 0))

# Reshape label array to (samples,)
y = label_array.ravel()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(-1, X_train.shape[1] * X_train.shape[2]), y_train)

# Compute class weights to handle imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)

# Model architecture
def cnn_model():
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = cnn_model()

# Model training with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2]), y_train_resampled, 
                    epochs=150, batch_size=64,
                    validation_data=(X_test, y_test),
                    class_weight=dict(enumerate(class_weights)),
                    callbacks=[early_stopping])

# Evaluate the model on the test data
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Epoch 1/150


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5146 - loss: 1.8920 - val_accuracy: 0.6054 - val_loss: 0.6853
Epoch 2/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5353 - loss: 0.6854 - val_accuracy: 0.6063 - val_loss: 0.6774
Epoch 3/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6036 - loss: 0.6616 - val_accuracy: 0.5580 - val_loss: 0.6841
Epoch 4/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.6652 - loss: 0.6287 - val_accuracy: 0.5404 - val_loss: 0.6913
Epoch 5/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7158 - loss: 0.5761 - val_accuracy: 0.5652 - val_loss: 0.6944
Epoch 6/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.7465 - loss: 0.5386 - val_accuracy: 0.5824 - val_loss: 0.7021
Epoch 7/150
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7848 - loss: 0.4803 - val_accuracy: 0.5919 - val_loss: 0.7531
70/70 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Test Accuracy: 0.6063205417607224
Classification Report:
      

In [6]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_points, channels)
X = np.transpose(data_array, (2, 1, 0))

# Reshape label array to (samples,)
y = label_array.ravel()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(-1, X_train.shape[1] * X_train.shape[2]), y_train)

# Define and train the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Model training
history = model.fit(X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2]), y_train_resampled, 
                    epochs=15, batch_size=32,
                    validation_data=(X_test, y_test), verbose=1)



# Evaluate the model on the test data
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 298, 64)        │         3,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 149, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 147, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 73, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 9344)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │       598,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 626,561 (2.39 MB)

 Trainable params: 626,561 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.4962 - loss: 1.0573 - val_accuracy: 0.3905 - val_loss: 0.6936
Epoch 2/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.5006 - loss: 0.6943 - val_accuracy: 0.3905 - val_loss: 0.6935
Epoch 3/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5016 - loss: 0.6941 - val_accuracy: 0.3905 - val_loss: 0.6935
Epoch 4/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.4998 - loss: 0.6932 - val_accuracy: 0.3905 - val_loss: 0.6934
Epoch 5/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4974 - loss: 0.6932 - val_accuracy: 0.3905 - val_loss: 0.6938
Epoch 6/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.4957 - loss: 0.6931 - val_accuracy: 0.3905 - val_loss: 0.6935
Epoch 7/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5027 - loss: 0.6931 - val_accuracy: 0.3905 - val_loss: 0.6933
Epoch 8/15
346/346 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.5079 - loss: 0.6931 - val_accu

C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_points, channels)
X = np.transpose(data_array, (2, 1, 0))

# Reshape label array to (samples,)
y = label_array.ravel()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build CNN model
def cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Experiment with different k_neighbors values in SMOTE
smote = SMOTE(k_neighbors=2, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.reshape(-1, X_train.shape[1] * X_train.shape[2]), y_train)

# Build and compile the CNN model
model = cnn_model()

# Train the model with the resampled data
history = model.fit(X_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2]), y_train_resampled, 
                epochs=15, batch_size=32,
                validation_data=(X_test, y_test), verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"k_neighbors=2, Test Accuracy: {accuracy}")


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6111 - loss: 0.7713
k_neighbors=2, Test Accuracy: 0.6022573113441467


In [13]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_points, channels)
X = np.transpose(data_array, (2, 1, 0))

# Reshape label array to (samples,)
y = label_array.ravel()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build CNN model
def cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Reshape input data to have rank 3
X_train_reshaped = X_train.reshape(-1, X_train.shape[1], X_train.shape[2])
X_test_reshaped = X_test.reshape(-1, X_test.shape[1], X_test.shape[2])

# Experiment with different k_neighbors values in SMOTE
smote = SMOTE(k_neighbors=2, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_reshaped.reshape(-1, X_train_reshaped.shape[1] * X_train_reshaped.shape[2]), y_train)

# Build and compile the CNN model
model = cnn_model()

# Train the model with the resampled data
history = model.fit(X_train_resampled.reshape(-1, X_train_reshaped.shape[1], X_train_reshaped.shape[2]), y_train_resampled, 
                epochs=15, batch_size=32,
                validation_data=(X_test_reshaped, y_test), verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f"k_neighbors=2, Test Accuracy: {accuracy}")


C:\Users\UC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6168 - loss: 1.0428
k_neighbors=2, Test Accuracy: 0.6158013343811035


In [20]:
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

# Load data from .mat file
file_path = 'C:\\Users\\UC\\Documents\\NeuMa\\22117124\\featuress.mat'
feet = sio.loadmat(file_path)
data_array = feet['data_array']
label_array = feet['label_array']

# Reshape data array to (samples, time_points, channels)
X = np.transpose(data_array, (2, 1, 0))

# Reshape label array to (samples,)
y = label_array.ravel()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build CNN model
def cnn_model(num_filters=64, kernel_size=3, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=num_filters*2, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the final CNN model with the best hyperparameters
best_model = cnn_model(num_filters=32, kernel_size=7, dropout_rate=0.3)

# Train the final model
best_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the final model
loss, accuracy = best_model.evaluate(X_test, y_test)
print("Final Test Accuracy:", accuracy)



Epoch 1/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5953 - loss: 0.9394 - val_accuracy: 0.6095 - val_loss: 0.6747
Epoch 2/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6265 - loss: 0.6589 - val_accuracy: 0.6099 - val_loss: 0.6731
Epoch 3/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6317 - loss: 0.6450 - val_accuracy: 0.6090 - val_loss: 0.6705
Epoch 4/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6367 - loss: 0.6340 - val_accuracy: 0.6050 - val_loss: 0.7138
Epoch 5/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6518 - loss: 0.6079 - val_accuracy: 0.6009 - val_loss: 0.6834
Epoch 6/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6773 - loss: 0.5785 - val_accuracy: 0.5968 - val_loss: 0.6980
Epoch 7/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7150 - loss: 0.5308 - val_accuracy: 0.6032 - val_loss: 0.7229
Epoch 8/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7613 - loss: 0.4755 - val_accu

Epoch 1/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6056 - loss: 0.7975 - val_accuracy: 0.6095 - val_loss: 0.6719
Epoch 2/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6183 - loss: 0.6676 - val_accuracy: 0.6090 - val_loss: 0.6728
Epoch 3/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6211 - loss: 0.6654 - val_accuracy: 0.6095 - val_loss: 0.6698
Epoch 4/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6227 - loss: 0.6640 - val_accuracy: 0.6095 - val_loss: 0.6712
Epoch 5/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6204 - loss: 0.6649 - val_accuracy: 0.6099 - val_loss: 0.6701
Epoch 6/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6225 - loss: 0.6645 - val_accuracy: 0.6095 - val_loss: 0.6726
Epoch 7/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6198 - loss: 0.6649 - val_accuracy: 0.6104 - val_loss: 0.6714
Epoch 8/100
277/277 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6222 - loss: 0.6633 - 